In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import itertools
import time
import concurrent.futures
import random

In [ ]:
def getSoup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    return soup


def getReviews(soup):
    user_review_list = soup.find_all('a', attrs={'class':'title'})
    
    user_review_tag = [review.attrs['href'] for review in user_review_list
                      if review.attrs['href'].startswith('/review') & review.attrs['href'].endswith('/')]
    
    user_review_tag = list(dict.fromkeys(user_review_tag))
    
    #user_review_link = "https://www.imdb.com" , user_review_tag
    
    user_review_link = ["https://www.imdb.com" + review_tag for review_tag in user_review_tag]
    
    return user_review_link


#def getReviewText(review_url):
    #soup = getSoup(review_url)
    #tag = soup.find('div', attrs={'class': 'text show-more__control'})
    #return tag.getText()
review = []
def getReviewText(review_url):
    soup = getSoup(review_url)
    for url in review_url:
        tag = soup.find('div', attrs={'class': 'text show-more__control'})
        review = tag.getText() if tag else "No review"
    return review
title = []
def getMovieTitle(review_url):
    soup = getSoup(review_url)
    for url in review_url:
        tag = soup.find('h1', attrs = {'class': "header"})
        tag = tag.find('a')
        title = tag.getText() if tag else "No title"
        #title = tag.children.getText()
    return title

In [ ]:
start = time.time()
movie_review_list = []
def getAllReviews():
    
    time.sleep(random.uniform(0, 0.2))
    url = "https://www.imdb.com/search/title/?title_type=feature&release_date=2014-01-01,2019-12-31&countries=gb&languages=en&count=10"

    movies_soup = getSoup(url)
    movie_tags = movies_soup.find_all('a', attrs={'class': None})

    movie_tags = [tag.attrs['href'] for tag in movie_tags
                 if tag.attrs['href'].startswith('/title') & tag.attrs['href'].endswith('/')]

    movie_tags = list(dict.fromkeys(movie_tags))

    #print("there are a total of " + str(len(movie_tags)) + " movie titles")
    #print('displaying 10 titles')

    base_url = "https://www.imdb.com"
    movie_links = [base_url + tag + 'reviews' for tag in movie_tags]
    #print("there are a total of " + str(len(movie_links)) + " movie user reviews")
    #print("displaying 10 user review links")
    #print(movie_links)
    
    movie_soups = [getSoup(link) for link in movie_links]
    movie_review_list = [getReviews(movie_soup) for movie_soup in movie_soups]

    movie_review_list = list(itertools.chain(*movie_review_list))
    print(len(movie_review_list))

    print("there are a total of " + str(len(movie_review_list)) + " individual movie reviews")
    print("displaying 10 reviews")

    return movie_review_list

getAllReviews()
movie_review_list = getAllReviews()
movie_titles = []
review_texts = []
m_title = []
r_text = []



with concurrent.futures.ThreadPoolExecutor() as executor:
    for url in movie_review_list:
        movie_titles.append(executor.submit(getMovieTitle, movie_review_list))
        
    for url in movie_review_list:
        review_texts.append(executor.submit(getReviewText, movie_review_list))
        
    for future in concurrent.futures.as_completed(review_texts):
        r_text.append(future.result())
    
    for future in concurrent.futures.as_completed(movie_titles):
        m_title.append(future.result())
    
end = time.time()

print("Time Taken: {:.6f}s".format(end-start))


In [ ]:
df = pd.DataFrame({'movie': m_title, "user_review_permalink": movie_review_list, 
                  'user_review': r_text})

In [ ]:
df.head()

In [ ]:
df.to_csv("userReviews.csv", index=False)